In [1]:
import urllib
import os
import re
import pandas as pd

In [2]:
filepath = 'html/python_class_test.html'
html_file = urllib.urlopen(filepath).read()

In [3]:
data = re.findall('(?<=<td>).*?(?=</td>)', html_file)

In [4]:
# removes the elements not ending with 'DAY' and adding it to 'just_color' array
def remove_days(data):
    just_colors = []
    for i in data:
        if i[-3: ] != 'DAY':
            just_colors.append(i)
    return just_colors
colors = remove_days(data)

In [5]:
# loops through a color-name-text and split them according to a matching pattern ", "
# after splitting a list created. Then i loop through the list adding each elements to a new list 
# which contains all the colors not in a text like form for a 'list of colors' form
def refactor_colors(colors):
    refactored_color_list = []
    for i in colors:
        split_elements = i.split(", ")
        for j in split_elements:
            refactored_color_list.append(j)
    return refactored_color_list
refactor_colors(colors)

just_colors = refactor_colors(colors)

In [6]:
# created a unique list of colors with out duplicates. Loops through them one after the other and adding them to 
# a dictionary with value = 0. checking if the name 
# exist in 'just_colors' - a list of all colors, anytime it finds a match its add one to the value in the dictionary

list_len_just_colors = len(just_colors)
unique_list = list(set(just_colors))
list_len_unique_list = len(unique_list)
def count_color(just_colors):
    color_dict = {}
    for index in range(0, list_len_unique_list):
        color_name = unique_list[index]
        color_dict[color_name] = 0
        for index2 in range(0, list_len_just_colors):
            if color_name  == just_colors[index2]:
                color_dict[color_name] +=1
        
                
        
    return color_dict
dictionary_of_colors_with_frequency = count_color(just_colors)

In [7]:
# converted the dictonary into a pandas object
# in other to calculate mean,variance,median
df = pd.DataFrame(dictionary_of_colors_with_frequency, index=[0])

In [8]:
df


,ARSH,BLACK,BLEW,BLUE,BROWN,CREAM,GREEN,"GREEN,",ORANGE,PINK,RED,WHITE,YELLOW
0,1,1,1,30,6,2,9,1,9,5,9,16,5


In [9]:
mean_color_number = round(df.mean(axis = 1).values, 0)
print mean_color_number
print 'closest mean color is Red'

7.0
closest mean color is Red


In [10]:
color_mostly_worn = df.max(axis = 1)
print color_mostly_worn.values
print 'color mostly worn is Blue'

[30]
color mostly worn is Blue


In [11]:
median_color = df.median(axis = 1)
print median_color.values
print "median color is YELLOw or PINK"

[ 5.]
median color is YELLOw or PINK


In [12]:
variance_of_color = df.var(axis = 1)
print "variance is " + str(variance_of_color.values)


variance is [ 66.56410256]


In [13]:
sum_of_all_colors = df.sum(axis = 1)
number_of_red = df['RED']
prob_of_red = number_of_red/sum_of_all_colors
print prob_of_red
print 'probability of picking a red color is ' +str(prob_of_red.values)

0    0.094737
dtype: float64
probability of picking a red color is [ 0.09473684]


## database

In [14]:
# database configuration code returns a dictionary of database, user, password and host
# which values will be used to connect to database

import psycopg2
import functools
from six.moves.configparser import ConfigParser

def config(filename='database.ini',section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1] # adding to dictionary here
    else:
        raise Exception('Section {0} not found in the {1} file'. format(section, filename))
    
    return db

/home/emeka/.local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [15]:
# using a python decorator to wrap around the query function. 
# This function connects to the database **params - contains the dictionary of credentials needed to acess postgresql
# query is executed and connection is closed
#create table decorator query runner
def create_table(query):
    @functools.wraps(query)
    def connect_run_close():
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            for sql in query():
                cur.execute(sql)
            conn.commit()
            msg = cur.statusmessage
            #print msg
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print error
        finally:
            if conn is not None:
                conn.close()
                #print('Database connection ended.')
    return connect_run_close


#table query
# sql query function create table with columns
@create_table #decorator definded above is called here
def create_table():
    query = ["""
            CREATE TABLE color_of_cloth(
            id SERIAL PRIMARY KEY,
            colors VARCHAR(100) NOT NULL,
            frequency VARCHAR(100) NOT NULL
            )
    """]
    return query


if __name__ == '__main__':
    create_table()


relation "color_of_cloth" already exists



In [16]:
#insert names into database  query runner
def insert_names_deco(query):
    @functools.wraps(query)
    def connect_run_close():
        conn = None
        try:
            params = config()
            conn = psycopg2.connect(**params)
            cur = conn.cursor()
            cur.execute(query())
            conn.commit()
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print error
        finally:
            if conn is not None:
                conn.close()
                
        return None
    return connect_run_close

In [17]:
#loops through the names of colors passing the names as key and getting the value. both are stored in the database
def add_rows_to_db():
    for i in unique_list:
        key = i
        value = dictionary_of_colors_with_frequency[i]
        @insert_names_deco
        def insert_query():
            query = """INSERT INTO color_of_cloth(colors,frequency) 
            VALUES('%s','%s') RETURNING id;""" % (key,value)
            return query
        insert_query()

add_rows_to_db()

### Fibonacci sequence

In [19]:
# fibonacci sequence
# keep adding the last two and appending the result
 # the appended result becomes the last - fibo[-1] and previous last
 # - fibo[-1] becomes second last ie fibo[-2]
def get_fibo_sequence():
    num = range(0,50)
    fibo = []
    new = 0
    for i in num:
        if not fibo:
            fibo.append(0)
        elif len(fibo) < 2:
            fibo.append(1)
        else:
            fibo.append(fibo[-2] + fibo[-1]) 
    return fibo

fibo_sequence = get_fibo_sequence()

sum_of_fibo = sum(fibo_sequence)
    
print sum_of_fibo
print fibo_sequence


20365011073
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049]
